# Naive predictors

We calculate naive predictors for the whole data set first in order to find out scores for these predictors on Kaggle platform and because there is no parameters to train for the chosen naive predictors.

Import libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime  
from datetime import timedelta  
import feather

Read data

In [ ]:
train_df = feather.read_dataframe('../01_preprocessed_data/train_df.feather')
calendar = feather.read_dataframe('../01_preprocessed_data/calendar.feather')
submission = pd.read_csv('../00_data/sample_submission.csv')

Determine the dates for prediction

In [ ]:
calendar.date[calendar['d'] == "d_1914"].iloc[0]

In [ ]:
start_pred_date = calendar.date[calendar['d'] == "d_1914"].iloc[0]
days_to_predict = 28
dates_to_predict = [start_pred_date + timedelta(days=x)  for x in range(days_to_predict)]
dates_to_predict

## Naive prediction 1

For each item take sales 364 days ago

In [ ]:
dates_last_year = [x - timedelta(days=(364))  for x in dates_to_predict ]

In [ ]:
naive_prediction_last_year = train_df[train_df.date.isin(dates_last_year)]

In [ ]:
naive_prediction_last_year

In [ ]:
predictions = pd.pivot(naive_prediction_last_year[['id', 'date', 'items_sold']], index = 'id', 
                       columns = 'date', values = 'items_sold').reset_index()

In [ ]:
predictions

In [ ]:
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

Evaluation rows are all zeros

In [ ]:
evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]
evaluation

In [ ]:
validation_rows = [row for row in submission['id'] if 'validation' in row] 
validation = submission[submission['id'].isin(validation_rows)]
validation = validation[['id']].merge(predictions, on = 'id', how = 'left')
validation


In [ ]:
final = pd.concat([validation, evaluation])
final = final.fillna(0)
final.to_csv("../04_submissions/naive_prediction_last_year.csv", index = False)

## Naive prediction 1

For each item take the average sold on the same day of the week

Add weekday to train_df

In [ ]:
weekly_means_df = train_df.groupby(['id','weekday'])['items_sold'].agg(['mean']).reset_index()

In [ ]:
weekly_means_df

In [ ]:
dates_to_predict_df = pd.DataFrame(dates_to_predict, columns =['date']) 
dates_to_predict_df = dates_to_predict_df.merge(calendar[['date', 'weekday']], on = 'date', how = 'left')

In [ ]:
dates_to_predict_df

In [ ]:
naive_prediction_weekly=submission[['id']]

In [ ]:
naive_prediction_weekly

In [ ]:
for i in range(0, 28):
    weekday = dates_to_predict_df.loc[i, 'weekday']
    weekly_means = weekly_means_df[weekly_means_df['weekday'] == weekday]
    weekly_means = weekly_means[['id', 'mean']]
    weekly_means = weekly_means.rename(columns={"mean": 'F'+str(i+1)})
    naive_prediction_weekly = naive_prediction_weekly.merge(weekly_means, on = 'id', how = 'left')

In [ ]:
naive_prediction_weekly = naive_prediction_weekly.fillna(0)

In [ ]:
naive_prediction_weekly

In [ ]:
naive_prediction_weekly.to_csv("../04_submissions/naive_prediction_weekly.csv", index = False)